In [ ]:
import random
import rospy
import jupyros
from sensor_msgs.msg import JointState
from joint_state_publisher import JointStatePublisher
from ipywidgets import FloatSlider, Layout, Button, Box, FloatText, Label

In [ ]:
rospy.init_node('niryo_node')

# Zethus

A robot description needs to be available (launch niryo.launch)

Features:
- RQT graph
- Info panel to view topic messages
- Other visualizations

In [ ]:
msg = JointState()
jsp = JointStatePublisher()
joint_state_dict = {}

for i in jsp.free_joints:
    minPos = jsp.free_joints[i]['min']
    maxPos = jsp.free_joints[i]['max']
    step = (maxPos - minPos) / 100
    joint_state_dict[i] = FloatSlider(min=minPos, max=maxPos, step=step)

### Joint State Publisher

In [ ]:
pub = rospy.Publisher('/joint_states', JointState, queue_size=10)

def on_sent(b):
    msg.header.stamp = rospy.Time.now()
    msg.name = [str(i) for i in joint_state_dict]
    msg.position = [joint_state_dict[i].value for i in joint_state_dict]
    pub.publish(msg)

for key, value in joint_state_dict.items():
    value.observe(on_sent)

In [ ]:
randomize = Button(description='Randomize')

def on_randomize(randomize):
    pub = rospy.Publisher('/joint_states', JointState, queue_size=10)
    for i in joint_state_dict:
        msg.name = [str(i)]
        msg.position = [random.uniform(joint_state_dict[i].min, joint_state_dict[i].max)]
        joint_state_dict[i].value = msg.position[0]
        pub.publish(msg)    
        
randomize.on_click(on_randomize)

In [ ]:
center = Button(description='Center')

def on_center(b):
    pub = rospy.Publisher('/joint_states', JointState, queue_size=10)
    for i in joint_state_dict:
        msg.name = [str(i)]
        msg.position = [0]
        joint_state_dict[i].value = 0
        pub.publish(msg)

center.on_click(on_center)

### Widget

In [ ]:
form_items1 = [Box([Label(value=j), joint_state_dict[j]]) for j in jsp.free_joints]
form_items1 += [Box([center]), Box([randomize])]

form = Box(form_items1, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='80%',
    padding='10px'
))

form

### Live Plots

In [ ]:
jupyros.live_plot('joint_states/:position', JointState)